# Match Chemical Fragments to Compounds

In [3]:
# read fragments dataset
import pickle
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from chembl_webresource_client.new_client import new_client
m=pickle.load(open("fragment_predictions.mols", "rb"))


In [4]:
#fragments matching to CHEMBL database
mols=[]
frags=[]
cluster=[]
for clu in range(0,11):
    for frag in range(len(m[clu])):
        smi=m[clu][frag]
        smi=Chem.MolToSmiles(smi,isomericSmiles=False)
        substructure = new_client.substructure
        res = substructure.filter(smiles=smi)
        try:
            m1= Chem.MolFromSmiles(res[0]['molecule_structures']['canonical_smiles'])
            print('ligand found for %s in cluster %d'%(smi,clu))
            
        except BaseException:
            print('cannot find ligand for %s in cluster %d'%(smi,clu))
            continue
        except len(res)==0:
            print('cannot find ligand for %s in cluster %d'%(smi,clu))
            continue    
            
        else:
            for i in range(0,11):
                for j in range(len(m[i])):
                    patt=m[i][j]
                    if m1.HasSubstructMatch(patt)==True:
                        mols.append(res[0]['molecule_structures']['canonical_smiles'])
                        frags.append(Chem.MolToSmiles(patt))
                        cluster.append(i)
                        print('%s in cluster %d' %(Chem.MolToSmiles(patt),i) )

cannot find ligand for O=CNCCc1ccccc1 in cluster 0
cannot find ligand for NCCNCc1ccccc1 in cluster 0
ligand found for NC(O)CNCc1ccccc1 in cluster 0
NCCNCc1ccccc1 in cluster 0
NC(O)CNCc1ccccc1 in cluster 0
CCNCc1ccccc1 in cluster 0
O=CCNCc1ccccc1 in cluster 0
OCCNCc1ccccc1 in cluster 0
CNCc1ccccc1 in cluster 0
NCc1ccccc1 in cluster 1
Cc1ccccc1 in cluster 9
ligand found for NS(=O)(=O)NCCc1ccccc1 in cluster 0
NS(=O)(=O)NCCc1ccccc1 in cluster 0
O=CCCc1ccccc1 in cluster 0
NCCc1ccccc1 in cluster 1
NCCCc1ccccc1 in cluster 1
NC(=O)CCc1ccccc1 in cluster 1
NC(=O)C(N)Cc1ccccc1 in cluster 6
CC(N)Cc1ccccc1 in cluster 6
Cc1ccccc1 in cluster 9
ligand found for CCNS(=O)(=O)Cc1ccccc1 in cluster 0
CCNS(=O)(=O)Cc1ccccc1 in cluster 0
Cc1ccccc1 in cluster 9
ligand found for CCS(=O)(=O)NCc1ccccc1 in cluster 0
CCS(=O)(=O)NCc1ccccc1 in cluster 0
CNCc1ccccc1 in cluster 0
NCc1ccccc1 in cluster 1
Cc1ccccc1 in cluster 9
cannot find ligand for O=CNCNCc1ccccc1 in cluster 0
cannot find ligand for CCNCc1ccccc1 in clu

ligand found for CCNN(C=O)c1ccccc1 in cluster 10
CCNN(C=O)c1ccccc1 in cluster 10
CNN(C=O)c1ccccc1 in cluster 10
cannot find ligand for CN(CC=O)c1ccccc1 in cluster 10
ligand found for CNN(C=O)c1ccccc1 in cluster 10
CCNN(C=O)c1ccccc1 in cluster 10
CNN(C=O)c1ccccc1 in cluster 10
ligand found for CCN(C=O)c1ccccc1O in cluster 10
CCN(C=O)c1ccccc1O in cluster 10
ligand found for N=CC(C=O)c1ccccc1 in cluster 10
CNCCc1ccccc1 in cluster 1
NCCc1ccccc1 in cluster 1
Cc1ccccc1 in cluster 9
N=CC(C=O)c1ccccc1 in cluster 10


In [6]:
import pandas as pd
result=pd.DataFrame({'matched ligands':mols,'fragments':frags,'clusters':cluster})

#add matching index in each matched group
num=0
index=[]
for i in range(len(mols)):
    index.append(num)
    if i!=len(mols)-1:
        if mols[i]==mols[i+1]:
            num+=1
        else:
            num=0
result['match index']=index
#result.to_csv('result.csv')

In [7]:
#delete compounds with only 1 matched fragment
#result=pd.read_csv('result.csv',index_col=0)
count=result.groupby(['matched ligands']).groups
for i in count.values():
    if len(i)==1:
        result.drop(index=i[0],inplace=True)
        print(i)
result=result.reset_index()

Int64Index([216], dtype='int64')
Int64Index([170], dtype='int64')
Int64Index([143], dtype='int64')
Int64Index([284], dtype='int64')


In [8]:
#add atom coverage

ratio=[]
for i in range(len(result['fragments'])):
    if result['match index'][i]==0:
        mol = Chem.MolFromSmiles(result['matched ligands'][i])
        patt = Chem.MolFromSmiles(result['fragments'][i])
        hit_ats1 = set(mol.GetSubstructMatch(patt))
        
        for j in range(1,14):
            if i+j>len(result['fragments'])-1 or result['match index'][i+j]==0:
                break
            else:
                mol = Chem.MolFromSmiles(result['matched ligands'][i+j])
                patt = Chem.MolFromSmiles(result['fragments'][i+j])
                hit_ats2 = set(mol.GetSubstructMatch(patt))
                hit_ats1=hit_ats1|hit_ats2
        #if len(hit_ats1)/mol.GetNumAtoms()>0.5:
        print(result['matched ligands'][i],len(hit_ats1)/mol.GetNumAtoms())
        ratio+=[len(hit_ats1)/mol.GetNumAtoms()]*(j)   

result['coverage']=ratio        
result=result[result['coverage']>0.5]
result.drop('index',axis=1,inplace=True)
result.reset_index(drop=True,inplace=True)
#result.sort_values(["coverage",'matched ligands','match index'],inplace=True)

CO[C@@]1(NC(=O)c2ccccc2)C(=O)N2C(C(=O)OC(c3ccccc3)c3ccccc3)=C(CSc3nnnn3C)CO[C@@H]21 0.3181818181818182
CCCOC(NC(=O)[C@H](Cc1ccccc1)NS(=O)(=O)N1CCOCC1)C(=O)N[C@@H](CC1CCCCC1)[C@@H](O)[C@@H](O)CC(C)C 0.35555555555555557
Nc1cc(C2=NOC(CNS(=O)(=O)Cc3ccccc3)(C(=O)Nc3ccc(-c4ccccc4S(N)(=O)=O)cn3)C2)ccc1Cl 0.45454545454545453
CCCCS(=O)(=O)N(Cc1ccccc1)c1ccc2ncn(Cc3ccc(-c4ccccc4-c4nnnn4C)cc3)c(=O)c2c1 0.3111111111111111
CC(C)c1cccc(C(C)C)c1N1CN(C(c2ccccc2)c2ccccc2)CC1=O 0.6451612903225806
C[C@](Cc1ccc(CC(=O)O)cc1)(NC(=O)[C@H](Cc1ccc(O)cc1)NNCc1ccccc1)C(=O)N[C@@H](CC(N)=O)C(N)=O 0.6444444444444445
CS(=O)(=O)C1CN(Cc2ccccc2)C(=O)C1=O 0.7777777777777778
C#CCOCN1c2ccccc2C(=O)N(Cc2ccccc2)S1(=O)=O 0.8
COC(=O)CCC1=CCN(NC(=O)c2ccccc2)CC1 0.5238095238095238
O=C1N(Cc2ccccc2)c2ccccc2C12OCCO2 0.8571428571428571
CC(NC(=O)C(=O)N/N=C1\C[C@H](c2ccccc2F)Nc2cc3c(cc21)OCO3)c1ccccc1 0.6571428571428571
O=C1NC(c2ccccc2)C2(CCN(CC(O)C3COc4ccccc4O3)CC2)O1 0.4
CCCCNC(=O)N[C@@H](Cc1ccccc1)C(=O)N[C@H](CC(C)C)C(=O)N[C@@H](Cc1

In [10]:
#add diversity
diversity=[]
for i in range(len(result['fragments'])):
    if result['match index'][i]==0:
        cluster=[]
        cluster.append(result['clusters'][i])
        for j in range(1,14):
            if i+j>len(result['fragments'])-1 or result['match index'][i+j]==0:
                break
            else:
                cluster.append(result['clusters'][i+j])
        if j!=len(cluster):
            print(i)
        div=len(set(cluster))
        diversity+=[div]*len(cluster)
        
result['diversity']=diversity

result.sort_values(["coverage",'matched ligands','match index'],inplace=True)
result.to_csv('preparation_for_label.csv',index=False)

In [13]:
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all" 

for i in range(len(result['fragments'])):
    mols=[]
    highlightAtomLists=[]
    highlightBondLists=[]
    legends=[]
    if result['match index'][i]==0:
        mol = Chem.MolFromSmiles(result['matched ligands'][i])
        patt = Chem.MolFromSmiles(result['fragments'][i])
        
        hit_ats = list(mol.GetSubstructMatch(patt))
        hit_bonds = []
        for bond in patt.GetBonds():
            aid1 = hit_ats[bond.GetBeginAtomIdx()]
            aid2 = hit_ats[bond.GetEndAtomIdx()]
            hit_bonds.append(mol.GetBondBetweenAtoms(aid1,aid2).GetIdx())
        
        atom=set(mol.GetSubstructMatch(patt))
        bonds=set(hit_bonds) 
        mols.append(mol)
        legends.append(result['fragments'][i]+' '+str(result['clusters'][i]))
        highlightAtomLists.append(hit_ats)
        highlightBondLists.append(hit_bonds)
        for j in range(1,14):
            if i+j>len(result['fragments'])-1 or result['match index'][i+j]==0:
                break
            else:
                mol = Chem.MolFromSmiles(result['matched ligands'][i+j])
                patt = Chem.MolFromSmiles(result['fragments'][i+j])
                hit_ats = list(mol.GetSubstructMatch(patt))
                hit_bonds = []
                for bond in patt.GetBonds():
                    aid1 = hit_ats[bond.GetBeginAtomIdx()]
                    aid2 = hit_ats[bond.GetEndAtomIdx()]
                    hit_bonds.append(mol.GetBondBetweenAtoms(aid1,aid2).GetIdx())
                    
                atom=atom|set(mol.GetSubstructMatch(patt))
                bonds=bonds|set(hit_bonds)
                mols.append(mol)
                legends.append(result['fragments'][i+j]+' '+str(result['clusters'][i+j]))
                highlightAtomLists.append(hit_ats)
                highlightBondLists.append(hit_bonds)
                
        mols.append(mol)
        legends.append('diverstity:'+ str(result['diversity'][i])+' coverage:'+str(round(result['coverage'][i],2)))
        highlightAtomLists.append(list(atom))
        highlightBondLists.append(list(bonds))
        img=Draw.MolsToGridImage(mols,molsPerRow=4,subImgSize=(200,200),highlightAtomLists=highlightAtomLists, highlightBondLists=highlightBondLists,legends=legends)    
        img.save('images/'+str(i)+'.png')